In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
fact_src_df = spark.read.format('parquet')\
    .option('inferschema',True)\
        .option('header',True)\
            .load('abfss://silver@dlsendtoendproject001.dfs.core.windows.net/silver_data')

display(fact_src_df)

**Creation of all the dimensions table dataframe required for join operations**

**Model Dimension Dataframe**

In [0]:
dim_model_df = spark.table('sales.gold.dim_model')
display(dim_model_df)

**Branch Dimension Dataframe**

In [0]:
dim_branch_df = spark.table('sales.gold.dim_branch')
display(dim_branch_df)

**Dealer Dimension Dataframe**

In [0]:
dim_dealer_df = spark.table('sales.gold.dim_dealer')
display(dim_dealer_df)

**Date Dimension Dataframe**

In [0]:
dim_date_df = spark.table('sales.gold.dim_date')
display(dim_date_df)

**Applying joins with fact tables of the above dimensions**

In [0]:
sales_fact_df = fact_src_df.join(dim_model_df, fact_src_df.Model_ID == dim_model_df.Model_ID,'left')\
    .join(dim_branch_df,fact_src_df['Branch_ID'] == dim_branch_df['Branch_ID'],'left')\
    .join(dim_dealer_df,fact_src_df['Dealer_ID'] == dim_dealer_df['Dealer_ID'],'left')\
    .join(dim_date_df,fact_src_df['Date_ID'] == dim_date_df['Date_ID'],'left')\
        .select(fact_src_df['Revenue'],fact_src_df['Units_Sold'],fact_src_df['RevenuePerUnit'],dim_branch_df['dim_branch_key'],dim_model_df['dim_model_key'],dim_dealer_df['dim_dealer_key'],dim_date_df['dim_date_key'])

display(sales_fact_df)

**Writing into fact Table**

In [0]:
if spark.catalog.tableExists('sales.gold.sales_fact'):
    deltaTable = DeltaTable.forPath(spark,'abfss://gold@dlsendtoendproject001.dfs.core.windows.net/sales_fact')
    deltaTable.alias('tgt').merge(sales_fact_df.alias('src'),'tgt.dim_branch_key = src.dim_branch_key AND tgt.dim_model_key = src.dim_model_key AND tgt.dim_dealer_key = src.dim_dealer_key AND tgt.dim_date_key = src.dim_date_key')\
        .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
                .execute()
else:
    sales_fact_df.write.format('delta')\
        .mode('overwrite')\
            .option('inferschema',True)\
                .option('path','abfss://gold@dlsendtoendproject001.dfs.core.windows.net/sales_fact')\
                    .saveAsTable('sales.gold.sales_fact')